# Audio Classification AI-ARM

In [ ]:
!pip install tensorflow-io

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import os

In [ ]:

## Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, calculate_results,walk_through_dir

## Download dataset

In [ ]:
# Setup enviroment variables for kaggle
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download dataset


In [ ]:
# Unzip data
#unzip_data("/content/drive/MyDrive/audio.rar")

In [ ]:
#!pip install unrar
!unrar x "/content/drive/MyDrive/audio.rar"

In [ ]:
walk_through_dir("audio")

In [ ]:
import shutil
for folders in os.listdir('audio'):
  for fol in os.listdir(f"audio/{folders}"):
    for file in os.listdir(f"audio/{folders}/{fol}"):
      shutil.move(f"audio/{folders}/{fol}/{file}", f"audio/{folders}/{file}")


## Create Tensorflow Dataset

In [ ]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="audio",
    batch_size=32,
    validation_split=0.2,
    seed=0,
    sampling_rate=48000,
    output_sequence_length=None,
    subset='both')

In [ ]:
label_names = np.array(train_ds.class_names)

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  #audio=audio[16000:]
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [ ]:
def get_spectograms(wav,label):
    #print(wav.shape)
    #wav = wav[16000:]
    #zero_padding = tf.zeros([16000] - tf.shape(wav), dtype=tf.float32)
    #wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=-1)
    #spectrogram = spectrogram[..., tf.newaxis]

    #print("shape")
    #print(spectrogram.shape)
    return spectrogram,label

In [ ]:
train = train_ds.map(map_func=get_spectograms,num_parallel_calls=tf.data.AUTOTUNE)
val = val_ds.map(map_func=get_spectograms,num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
for sample,label in train.take(1):
    print(sample.shape)
    print(label.shape)

## Preprocess dataset

In [ ]:
#lengths = []
#for folders in os.listdir('audio'):
#  for fol in os.listdir(f"audio/{folders}"):
#    for file in os.listdir(f"audio/{folders}/{fol}"):
#      tensor_wave = load_wav_16k_mono(f"audio/{folders}/{fol}/{file}")
#      lengths.append(len(tensor_wave))

In [ ]:
#tf.math.reduce_mean(lengths)

In [ ]:
#tf.math.reduce_min(lengths)

In [ ]:
#tf.math.reduce_max(lengths)

### Building preprocessing Function

### Test Out the Function and Viz the Spectrogram


## Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras import layers

In [ ]:
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
num_labels = len(label_names)

norm_layer.adapt(data=train.map(map_func=lambda spec, label: spec))

model = Sequential([
    layers.Input(shape=(4264,257,1)),
    # Downsample the input.
    layers.Resizing(64, 64),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),

    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),

    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
model.summary()

In [ ]:
# Fit model
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=20,restore_best_weights=True)
hist = model.fit(train, epochs=150,validation_data=val,callbacks=[earlyStopping])

In [ ]:
model.save('/content/drive/MyDrive/savedmodel')

In [ ]:
model.save('/content/drive/MyDrive/model.keras')